In [1]:
import os
import gradio as gr
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


e:\assignment\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY", "")


In [ ]:
# api keys
groq_api_key = ""
cohere_api_key = ""


In [ ]:
# sample data
RAW_FAQS = [
    {"question": "What are your company's working hours?",
     "answer": "Our company operates from 9 AM to 5 PM, Monday to Friday."},
    {"question": "How can I contact customer support?",
     "answer": "Email support@example.com or call 1-800-123-4567."},
    {"question": "What products do you offer?",
     "answer": "We offer cloud platforms, data analytics, and cybersecurity solutions."},
    {"question": "Where is your main office located?",
     "answer": "123 Tech Avenue, Innovation City, CA 90210."},
    {"question": "What is your return policy?",
     "answer": "Full refunds within 30 days if product is in original condition."},
    {"question": "Do you offer international shipping?",
     "answer": "Yes, shipping cost and delivery times vary by location."},
    {"question": "What payment methods do you accept?",
     "answer": "Visa, MasterCard, AmEx, PayPal, and bank transfers."},
    {"question": "What is your company's mission statement?",
     "answer": "Empowering businesses with innovative technology solutions."},
    {"question": "How can I apply for a job?",
     "answer": "Visit our careers portal on our website."},
    {"question": "Do you provide training for your software?",
     "answer": "Yes, including online tutorials, webinars, and workshops."},
]


In [ ]:
# json to doc
documents = []
for i, faq in enumerate(RAW_FAQS):
    content = f"Question: {faq['question']}\nAnswer: {faq['answer']}"
    documents.append(Document(page_content=content, metadata={"source": f"faq_{i}"}))


In [ ]:
# chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)


In [ ]:
# create embeddings
embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-english-v3.0"
)

# store in vector db
vectorstore = FAISS.from_documents(chunks, embeddings)


In [ ]:
# llm model
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    groq_api_key=groq_api_key,
    max_tokens=150
)


In [ ]:
# prompt template for better results
prompt_template = PromptTemplate.from_template("""
You are a helpful assistant. Use the following company FAQs to answer the user's question in a detailed, friendly, and conversational manner. Maintain awareness of the previous chat if relevant.

Context:
{context}

Current Question: {question}
Answer:""")


In [ ]:
# saving history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer' 
)




C:\Users\hp\AppData\Local\Temp\ipykernel_15252\2981587399.py:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
# chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    return_source_documents=True
)


In [ ]:

chat_history = []


In [11]:
user_input = "What services do you provide ?"


In [12]:
result = rag_chain.invoke({
            "question": user_input,
            "chat_history": chat_history  # maintained by LangChain memory
        })

answer = result["answer"]
sources = result.get("source_documents", [])
context = "\n\n".join([f"📄 **Doc {i+1}:** {doc.page_content}" for i, doc in enumerate(sources)])


In [13]:
print("Response: ", answer)
print("Context: ", sources)


Response:  You're looking to know more about the services we provide, right?

Well, I'm happy to tell you that our services are closely tied to the products we offer. Since we provide cloud platforms, data analytics, and cybersecurity solutions, our services are designed to help you get the most out of these products.

For instance, we offer implementation services to help you set up and integrate our cloud platforms into your existing infrastructure. Our team of experts will work closely with you to ensure a seamless transition.

When it comes to data analytics, our services include data migration, data warehousing, and business intelligence solutions. We can help you make sense of your data and turn it into actionable insights that drive business growth.

In terms of cybersecurity, our services include threat
Context:  [Document(id='ad2d6dc9-6a0a-4fc1-b252-e593264aba80', metadata={'source': 'faq_2'}, page_content='Question: What products do you offer?\nAnswer: We offer cloud platform